<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## 1. Scraping the web to obtain relevant data

As mentioned, the data for the toronto neighbourhoods is not readily available in the form of a pre eisting file. Instead the relevant data is present in the form of a table on a wikipedia page. Thus in order to proceed with our objective we first need to apply various web scraping techniques and extract that data from wikipedia followed by which we need to wrangle and clean it so that it appears in the right format. After the aforementioned steps are over, we can start processing and clustering the data using forsquare API.

We begin by using the urllib.request library to fetch all the html data from the webpage

In [1]:
#the urllib.request library helps us to request for and obtain data resources present on a specified webpage
import urllib.request
#the wikipedia page with the required table
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#creating a request object and using the urlopen() function of the library to obtain data
page =urllib.request.urlopen(url)
#the variable page now contains all the data in the html format from the webpage specified by the above link

We now import the beautiful soup library and use it to parse the html data present in the page variable from the above step. To get an idea about the structure of the html present inthe webpage we use the prettify() method of the beautiful soup library and print out the results.

In [4]:
!pip install bs4
from bs4 import BeautifulSoup as bb
soup = bb(page, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"f1e4cccf-e978-4db8-b92c-e9bb8fd65424","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":969510799,"wgRevisionId":969510799,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

We now use the find_all() method of the beautiful soup library to find all the instances of the table tag in the html code of the given table. Which means that we ae trying to obtain. alist of all the tables on the webpage so that we can extract the relevant table out of all present.

In [5]:
all_tables=soup.find_all('table')
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B

when we look through the list of all tables present in the all_tables variables, we realize that the class of the required table is 'wikitable sortable'. Thus we use the find() method of the beautiful soup library to get the table we want and store its contents in a variable calle dthe right table. The data, as can be seen by running the code cell below, is still in the html format.

In [6]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

If the results obtained from the code cell abive are analysed, it can be seen that the tr tag corresponds to the entries of an entire row and the tag td to that of one cell in the table. This we use the findAll() method again to obtain the rows of the table and use a for loob to create lists that store the data corresponding to each column of the table. There are three rows in the required table.

In [7]:
#empty lists to store the data of each column.
A=[]
B=[]
C=[]
#at each iteration, the variable row will have the data corresponding to one row of the table.
for row in right_table.findAll('tr'):
    #cells is a list of all three cells of the row.
    cells=row.findAll('td')
    if len(cells)==3:
        #appending the assosciated columns to each list
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))      

We now use the pandas library to create a data frame using the data obtained in the three lists A,B, and C above.

In [8]:
import pandas as pd
import numpy as np
df=pd.DataFrame(A,columns=['Postal code'])
df['Borough']=B
df['Neighbourhood']=C
print(df.shape)
df.head()

(180, 3)


,Postal code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Thus we now have a raw data frame with the required data. The next task is to clean and wrangle it.

## 2. Cleaning and Wrangling data to obtain optimum data frame

First we begin by processing the data frame and excluding all the columns that have an unassigned burough.

In [9]:
# we start by creating a new data frame called toronto data 
toronto_data=pd.DataFrame()
#then we assign all those column values to this data frame which have a valid borough 
toronto_data=df.drop(df[df.Borough=='Not assigned\n'].index)
#we view the newly created data frame
print(toronto_data.shape)
toronto_data.head()

(103, 3)


,Postal code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


with the help of the df.shape operation we can conclude that about 77 entries had invalid borough values and were thus excluded from the new data frame. We then go on to reset the index of our data frame to compensate for the missing values.

In [10]:
toronto_data.reset_index(inplace = True, drop = True)
toronto_data.head()

,Postal code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now we shall deal with all the neighbourhoods that have a not assigned value and give them the same value as their corrresponding borough.

In [11]:
data=toronto_data.shape[0]
#data ow has the total number of rows in our data drame
#we run a loop so that it runs from 0 to the total number of rows in the data frame so that each row can be examined seperately.
for num in range(data):
    row=list(toronto_data.iloc[num,:])
    # we extract the contents of each row as a list and then check the 2nd index which corersponds to the neighbourhoods column and see if its
    #not assigned or not.
    if row[2]=='Not assigned\n':
        row[2]=row[1]
        #if the neighbourhood is not assigned then the 1st index is assigned as its value; i.e the borough

In [12]:
# we now print the shape of the data frame
toronto_data.shape

(103, 3)

## 3. Using geocoder library to extract the longitude and latitudes of each of the postal codes

we begin by importing all of the relevant modules

In [14]:
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
ge

Now that we have imported the relevant libraries, we will parse each zip code into the geocode() method of the geopy library to get the longitute and latitude values for each neighbourhood. After this, we will add longitude latitute values to our data frame to create a new table with all the required clustering information.

In [15]:
#!conda install -c conda-forge geocoder --yes
!pip install pgeocode
import pgeocode # import geocoder
# initialize your variable to None
lat_list=[]
long_list=[]
for num in range(data):
    row=list(toronto_data.iloc[num,:])
    postal_code=row[0]
    obj=pgeocode.Nominatim('ca')
    result=obj.query_postal_code([postal_code])
    lat=result.latitude
    long=result.longitude
    lat_list.append(lat[0])
    long_list.append(long[0])

In the above block of code the lists lat_list and long_list now have all the required coordinates. the next step is to add these lists to the toronto_data dataframe as columns.

In [16]:
toronto_data['Latitude']=lat_list
toronto_data['Longitude']=long_list
toronto_data.head()

,Postal code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


The above data frame is the optimum that is required. We will now begin with clustering.

## 3. Segmenting and clustering the data


Here we will use the created foursquare api to access the data base and extract relevant information about the nearby venues in each neighbourhood of toronto. We will then manipulate this data and perform one hot encoding and extract the top 10 most popular venues for clustering and plotting purposes

First we begin by defining our fourquare credentials and generating the request link.

In [17]:
# Defining the client credentials:
CLIENT_ID = '5PUZKJEJVILJGWCEPGIJYXD45BZ1XB4NO40JKYBTS5YKGEHB' # your Foursquare ID
CLIENT_SECRET = 'UUNGM1ARCVHGD5L4VKN1GCYHWA2KX5X12OXHYGLEFTZ2WCGW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5PUZKJEJVILJGWCEPGIJYXD45BZ1XB4NO40JKYBTS5YKGEHB
CLIENT_SECRET:UUNGM1ARCVHGD5L4VKN1GCYHWA2KX5X12OXHYGLEFTZ2WCGW


Now we shall define a function that will extract the latitude and longitude of each neighbourhood from the toronto_data data frame and get the venues around that area.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    limit=100
    radius=500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
        try:    
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            continue
                

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We will now write code that helps us extract the relevant data and store it in a data frame called toronto venues

In [19]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [20]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.7545,-79.3300,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant


As can be seen from the result obtained above, the data frame has entries pertaining to the venues that were found in the foursquare api near or in the vicinity of a given neighbourhood. We will now perform one hot encoding based on the 'Venue category' column of the toronto_venues data frame.

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Since the above table has multiple entries for a singke neighbourhood, we shall now recompile the data and obtain a new data frame called toronto_grouped which has the average values for all venue categories for one neighbourhood as a single row entry

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


based on the table obtained above, we create a new table or data frame calld the neighbourhoods venue sorted wherin we will have the neighbourhood name followed by the 10 most popular venue categories there lsited as 10 columns. For this purpose we define a function called return_most_common_venues().

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Skating Rink,Badminton Court,Cupcake Shop,Electronics Store,Flower Shop,Flea Market,Fish Market,Creperie
1,"Alderwood, Long Branch",Pizza Place,Gym,Dance Studio,Pub,Coffee Shop,Sandwich Place,Convenience Store,Pharmacy,Gym / Fitness Center,Fast Food Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Deli / Bodega,Fried Chicken Joint,Middle Eastern Restaurant,Coffee Shop,Mediterranean Restaurant,Dance Studio,Curling Ice,Flea Market,Fish Market
3,Bayview Village,Flower Shop,Gas Station,Trail,Park,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Restaurant,Coffee Shop,Indian Restaurant,Café,Liquor Store,Boutique,Pub,Sushi Restaurant


Now that we have the above table we first drop the column with rhe neighbourhood's names in it so that a dataframe with only clustering relevant information remains. we then used the k means clustering algorithm to clster the neighbourhoods and obtain cluster labels. For this purpose we choose the value of k as 4.

In [24]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 3, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
       1, 1, 3, 1, 1, 3, 1], dtype=int32)

we now create a new data frame called toronto merged wherin we merge the two tables: toronto_data and neighbourhoods_venues_sorted. In addition to this we add another column indicating the cluster labels of each neighbourhood. Since there are some rows wherin nan values have accumilated due to failed longittude/latitude value extractions, we shall also tackle such entries.

In [25]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels']=kmeans.labels_

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.dropna(axis=0,how='any',inplace=True)
toronto_merged.reset_index(inplace=True,drop=True)
toronto_merged

,Postal code,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M3A,North York,Parkwoods,43.7545,-79.3300,Food & Drink Shop,Construction & Landscaping,Park,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Field,1.0
1,M4A,North York,Victoria Village,43.7276,-79.3148,Pizza Place,Coffee Shop,Intersection,French Restaurant,Park,Portuguese Restaurant,Hockey Arena,Art Gallery,Food Truck,Food & Drink Shop,1.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Spa,1.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,Clothing Store,Coffee Shop,Women's Store,Restaurant,Sushi Restaurant,Toy / Game Store,Men's Store,Jewelry Store,Food Court,Bakery,1.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Coffee Shop,Bubble Tea Shop,Theater,Martial Arts Dojo,Burrito Place,Mexican Restaurant,Ethiopian Restaurant,Ramen Restaurant,Beer Bar,Sushi Restaurant,1.0
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282,Pharmacy,Bank,Grocery Store,Park,Skating Rink,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,0.0
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930,Home Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,2.0
7,M3B,North York,Don Mills,43.7450,-79.3590,Park,Gym,Home Service,Golf Driving Range,Pool,Trail,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,1.0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,Pizza Place,Gym / Fitness Center,Gastropub,Intersection,Bank,Café,Pet Store,Pharmacy,Fast Food Restaurant,Farmers Market,1.0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,Coffee Shop,Clothing Store,Café,Hotel,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Lingerie Store,Bubble Tea Shop,Ramen Restaurant,1.0


For the purpose of plotting data we use the geocoder library to obtain the coordinates of the city of toronto

In [26]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


We now use the folium package to visulaize our clusters.

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(7)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)
    
map_clusters


In [33]:
neigh= input("Enter the name of the neighborhood you currently reside in...")
ans=neigh+'\n'
similar=[]
loca=list( toronto_merged['Neighbourhood'])
label=list(toronto_merged['Cluster Labels'])
for name,labe in zip(loca,label):
    if name==ans:
        clustno=labe
for name,labe in zip(loca,label):
    if name==ans:
        continue
    if labe==clustno:
        similar.append(name.strip())
print("The top neighborhoods similar to yours are...")
similar=list(dict.fromkeys(similar))
length=len(similar)
for num in range(length):
    print(num+1,". ",similar[num])
    print("\n")

Enter the name of the neighborhood you currently reside in... Parkwoods


The top neighborhoods similar to yours are...
1 .  Victoria Village


2 .  Regent Park, Harbourfront


3 .  Lawrence Manor, Lawrence Heights


4 .  Queen's Park, Ontario Provincial Government


5 .  Don Mills


6 .  Parkview Hill, Woodbine Gardens


7 .  Garden District, Ryerson


8 .  Glencairn


9 .  West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale


10 .  Woodbine Heights


11 .  St. James Town


12 .  Humewood-Cedarvale


13 .  Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood


14 .  Guildwood, Morningside, West Hill


15 .  The Beaches


16 .  Berczy Park


17 .  Caledonia-Fairbanks


18 .  Woburn


19 .  Leaside


20 .  Central Bay Street


21 .  Christie


22 .  Cedarbrae


23 .  Bathurst Manor, Wilson Heights, Downsview North


24 .  Thorncliffe Park


25 .  Richmond, Adelaide, King


26 .  Dufferin, Dovercourt Village


27 .  Scarborough Village


28 .  Fairview, Henry Farm, Oriole


29 .  Northwood Park, York University


30 .  East Toronto,